<a href="https://colab.research.google.com/github/Llmortimer/character_morality/blob/main/DissertationFilmAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install -y ffmpeg

import re
import string
from difflib import SequenceMatcher
from operator import itemgetter
from types import CoroutineType


input_script = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeScript.txt"
input_subtitles = "/content/drive/MyDrive/shared/Scripts_and_Films/Indiana Jones And The Last Crusade DVDRip Xvid -IZON-.srt"
dialogue_script = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeDialogue.txt"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
#pre-process subtitles by extractings start and end times of each dialogue.
f = open(input_subtitles).read()
matches = re.findall(r"([0-9]*)\n(.+)--> (.+)\n((.+\n{0,1})+)", f, re.M)
id = []
start = []
end = []
dialogue_sub = []
for i in matches:
    id.append(i[0])
    start.append(i[1])
    end.append(i[2])
    dialogue_sub.append(i[3].rstrip().replace('\n', ' '))

In [83]:
#pre-process script by extracting the dialogue and character names writing to seperate file.
f = open(input_script).read()
matches = re.findall(r"\n\s+([A-Z ]+)\n *((.+\s)*)", f, re.M)
dialogue = []
characters = []
for i in matches:
    if '' not in i:
        j = " ".join(i[1].split())
        j = j.split(".")
        while '' in j:
            j.remove('')

        for k in j:
            k.strip()
            dialogue.append(k)
            characters.append(i[0])

        

with open(dialogue_script, "w") as ds:
    for inst in dialogue:
        ds.write(inst)

In [ ]:
print(len(dialogue))
print(len(dialogue_sub))

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

totals = []
totals_ids = []
combined = []
combined_dialogue = []
recent = -1
last = ''
id_max = -1

for c, i in enumerate(dialogue_sub):
    totals = []

    for count, j in enumerate(dialogue):
        if count < (recent + 10) and count > recent:
            totals.append((similar(i.translate(str.maketrans('', '', string.punctuation)), j.translate(str.maketrans('', '', string.punctuation))), c, count, i, j))
        # if similar(i, j) > 0.7 and recent < count and :
        #     combined.append([id[c], count])
        #     combined_dialogue.append([i, j])

        #     recent = c
        #     break

    id_max = totals.index(max(totals, key=itemgetter(0)))
    recent = max(0, totals[id_max][2]-3)
    combined.append(totals[id_max])
    

for c in combined:
    print(c)

# print(combined_dialogue)


In [79]:
print(len(dialogue))
print(len(dialogue_sub))

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

totals = []
totals_ids = []
combined = []
combined_dialogue = []
recent = -1
last = 0
id_max = -1

for c, i in enumerate(dialogue_sub):
    for count, j in enumerate(dialogue):

        if similar(i, j) > 0.7 and count > last and count < last + 10:
            combined.append((similar(i, j), c, count, i, j))
            last = count
            break

list1 = []
list2 = []

last_sub = -1
last_script = -1

for c in combined:
    if c[1] > last_sub+1:
        list1.append(list(range(last_sub+1, c[1])))
    
    list1.append([c[1]])
    last_sub = c[1]

    if c[2] > last_script+1:
        list2.append(list(range(last_script+1, c[2])))

    list2.append([c[2]])
    last_script = c[2]

list1.append(list(range(last_sub+1, len(dialogue_sub))))
list2.append(list(range(last_script+1, len(dialogue))))

In [80]:
for c in combined:
    print(c)

(0.9743589743589743, 1, 1, "Herman's horse-sick!", "Herman's horsesick!")
(0.7142857142857143, 2, 2, 'Chaps, no one wander off.', "Chaps, don't anybody wander off")
(0.98, 3, 3, 'Some of the passageways in here can run for miles.', ' Some of the passageways in here can run for miles')
(0.961038961038961, 4, 4, "l don't think this is such a good idea.", "I don't think this is such a good idea")
(1.0, 5, 5, 'What is it?', 'What is it?')
(1.0, 6, 6, 'Alfred, did you get anything yet?', 'Alfred, did you get anything yet?')
(0.9333333333333333, 7, 7, 'Nothing.', 'Nothing')
(0.75, 9, 9, "The kid's got something!", "Hey, I've got something!")
(0.9565217391304348, 11, 10, 'l got something, Garth!', 'I got something, Garth!')
(0.7123287671232876, 12, 12, 'l got something... l got something right here.', ' I got something right here')
(0.8888888888888888, 22, 17, 'Shh!', 'Shhh!')
(0.9887640449438202, 23, 18, 'Hey, we got to find more stuff to bring back.', 'Hey, we got to find more stuff to brin

In [89]:
char_annotated = []
print(characters)

for srt, sct in zip(list1, list2):
    char = []
    dia = []
    for i in sct:
        char.append(characters[i])
        dia.append(dialogue[i])
        
    char_annotated.append((char, srt, sct, dia, start[srt[0]], end[srt[-1]]))

for i in char_annotated:
    print(i)

['OFFICER', 'FIRST SCOUT', 'SCOUTMASTER', 'SCOUTMASTER', 'HERMAN', 'HERMAN', 'FEDORA', 'MAN', 'MAN', 'ROSCOE', 'ROSCOE', 'ROSCOE', 'ROSCOE', 'ROSCOE', 'HALFBREED', 'HALFBREED', 'HERMAN', 'INDY', 'HALFBREED', 'INDY', 'INDY', 'INDY', 'INDY', 'INDY', 'INDY', 'INDY', 'INDY', 'HERMAN', 'HERMAN', 'HERMAN', 'HERMAN', 'HERMAN', 'INDY', 'INDY', 'FEDORA', 'ROSCOE', 'HALFBREED', 'INDY', 'FEDORA', 'MEN', 'INDY', 'ROSCOE', 'PANAMA HAT', 'PANAMA HAT', 'INDY', 'INDY', 'INDY', 'ROUGH RIDER', 'INDY', 'FEDORA', 'FEDORA', 'INDY', 'FEDORA', 'FEDORA', 'FEDORA', 'INDY', 'FEDORA', 'FEDORA', 'INDY', 'ROSCOE', 'FEDORA', 'INDY', 'FEDORA', 'FEDORA', 'FEDORA', 'FEDORA', 'FEDORA', 'FEDORA', 'INDY', 'INDY', 'INDY', 'HENRY', 'INDY', 'HENRY', 'INDY', 'INDY', 'HENRY', 'INDY', 'HENRY', 'INDY', 'INDY', 'INDY', 'INDY', 'HENRY', 'HENRY', 'HERMAN', 'INDY', 'SHERIFF', 'INDY', 'INDY', 'INDY', 'SHERIFF', 'INDY', 'INDY', 'SHERIFF', 'SHERIFF', 'SHERIFF', 'ROSCOE', 'SHERIFF', 'FEDORA', 'PANAMA HAT', 'INDY', 'PANAMA HAT', 'INDY',

In [ ]:
#scene detection, for each frame assign a probablility that the scene has changed given the change of intensity of the pixels on the pervious frame.
#if this probablility is higher than 0.5 for a given frame the frame time, and probablilty have been output to a file "time.txt" and the frame itself output to a PNG in scenes/ 

!ffmpeg -i /content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeFilm.mp4 -filter_complex "select='gt(scene,0.5)',metadata=print:file=/content/drive/MyDrive/shared/scenes/time.txt" -vsync vfr /content/drive/MyDrive/shared/scenes/img%03d.png

In [ ]:
#extract the scene changes and their times from the video file.